In [1]:
template = {
    'user_id': int,
    'name': str,
    "address": {
        "street": str,
        "house": int,
        "location": {
            "district": str,
            "idx": int
        }
    }
}

olia = {
    "user_id": 1,
    "name": "olia",
    "address": {
        "street": "Mircea",
        "house": 56,
        "location": {
            "district": "calea",
            # "idx": 2043
        }
    }
}

In [3]:
def validate(data, template, is_recursive, missing_from_data=None, missing_from_template=None):
    if missing_from_data is None:
        missing_from_data = ""
    if missing_from_template is None:
        missing_from_template = ""
    
    
    if data.keys() ^ template.keys() != set():
        missing_field = (template.keys() - data.keys())
        if missing_field:
            missing_from_data += "." + missing_field.pop()
            if not is_recursive:
                print("The following data is missing: template"+missing_from_data)
        else:
            missing_from_template += "." + (data.keys() - template.keys()).pop()
            if not is_recursive:
                print(f"The template does not have" + " template"+missing_from_template)
                
        return False, missing_from_data, missing_from_template

    else:
        for k, v in template.items():
            if isinstance(v, dict):
                inter_res, missing_from_data, missing_from_template = validate(data=data[k], template=v, is_recursive=True, missing_from_data=missing_from_data, missing_from_template=missing_from_template)
                if not inter_res:
                    if missing_from_data:
                        missing_from_data = "." + k + missing_from_data
                    elif missing_from_template:
                        missing_from_template = "." + k + missing_from_template
                    if not is_recursive:
                        if missing_from_template:
                            print(f"The template does not have: template" + missing_from_template)
                        
                        elif missing_from_data:
                            print("The following data is missing: template" + missing_from_data)
                            
                    return False, missing_from_data, missing_from_template
                
            elif not isinstance(data[k], v):
                print("======================")
                print(f"Expected: {k, v}\nactual: {k, type(data[k])}")
                print("======================")
                return False, missing_from_data, missing_from_template
      
    return True, missing_from_data, missing_from_template
                


validate(olia, template, False)

The following data is missing: template.address.location.idx


(False, '.address.location.idx', '')

In [110]:
class SchemaError(Exception):
    pass

class SchemaKeyMismatch(SchemaError):
    pass

class SchemaTypeMismatch(SchemaError, TypeError):
    pass

In [129]:
template = {
    'user_id': int,
    'name': str,
    "address": {
        "street": str,
        "house": int,
        "location": {
            "district": str,
            "idx": int
        }
    }
}

olia = {
    "user_id": 1,
    "name": 'olga',
    "address": {
        "street": "Mircea",
        "house": 56,
        "location": {
            "district": "calea",
            "idx": "hey"
        }
    }
}

In [142]:
def validate(data, template, path="."):

    if data.keys() ^ template.keys() != set():
        if (data.keys() - template.keys()):
            raise SchemaKeyMismatch(f"Key is not supported: {path}{(data.keys() - template.keys()).pop()}")
        else:
            raise SchemaKeyMismatch(f"Missing key: {path}{(template.keys() - data.keys()).pop()}")
            
    else:
        for k, v in template.items():
            if isinstance(v, dict):
                sub_path = path + str(k) + "."
                validate(data[k], template[k], sub_path)

            elif not isinstance(data[k], v):
                raise SchemaTypeMismatch(f"\nExpected: {path}{k}: {v}\nActual:   {path}{k}: {type(data[k])}   ('{data[k]}')")

In [143]:
validate(olia, template)

SchemaTypeMismatch: 
Expected: .address.location.idx: <class 'int'>
Actual:   .address.location.idx: <class 'str'>   ('hey')

In [144]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

eric = {
    'user_id': 101,
    'name': {
        'first': 'Eric',
        'last': 'Idle'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 3,
            'day': 29
        },
        'birthplace': {
            'country': 'United Kingdom'
        }
    }
}

validate(data=eric, template=template)

SchemaKeyMismatch: Missing key: .bio.birthplace.city

In [145]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}

validate(data=john, template=template)

In [146]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

eric = {
    'user_id': 101,
    'name': {
        'first': 'Eric',
        'last': 'Idle'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 3,
            'day': 29
        },
        'birthplace': {
            'country': 'United Kingdom'
        }
    }
}

validate(data=eric, template=template)

SchemaKeyMismatch: Missing key: .bio.birthplace.city

In [147]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

michael = {
    'user_id': 102,
    'name': {
        'first': 'Michael',
        'last': 'Palin'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 'May',
            'day': 5
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Sheffield'
        }
    }
}

validate(data=michael, template=template)

SchemaTypeMismatch: 
Expected: .bio.dob.month: <class 'int'>
Actual:   .bio.dob.month: <class 'str'>   ('May')